# PAYMUL generation


In [1]:
import xlwt, xlrd

## Data.xlsx import

In [2]:
# Open an Excel file. (xlrd object)
wb = xlrd.open_workbook('Data.xlsx')

# How many sheets in the file? What are their names?
print(wb.sheet_names())

# An Excel spreadsheet, like an Excel fils, is an xlrd object.
sh = wb.sheet_by_name('Data')

['Data', 'Output']


## Excel file reading

In [3]:
# how many rows and columns in this sheet?
print('nrows = ',sh.nrows, 'ncols = ', sh.ncols)

# Transfert lists creation
payeeslist = sh.col_values(0)
amountlist = sh.col_values(1)
banklist = sh.col_values(2)
datatransfert = sh.col_values(5)

# Values transferts sum   
transfertsum = str(round(sum(amountlist[1:]),2))

# Transfert data creation
datatransfertdict = {}
datatransfertdict_entries = ['payref', 'paydate', 'paynotes', 'payacc', 'uniqref', 'today']

for i in range(len(datatransfertdict_entries)):
    datatransfertdict[datatransfertdict_entries[i]] = datatransfert[i]

for i in datatransfertdict:
    if type(datatransfertdict[i]) != str:
        datatransfertdict[i] = str(int(datatransfertdict[i]))
        
# Payment account 
payaccount = datatransfertdict['payacc']
payaccsort = payaccount[:8]
payaccname = payaccount[9:-10]
payaccnum = payaccount[-6:]

nrows =  59 ncols =  6


## Excel file controls

## Paymul file generation

In [4]:
# Create an Excel file (xlwt object)
outputbook = xlwt.Workbook()

# Create an Excel spreadsheet
paymuldata = outputbook.add_sheet('Paymuldata',True) #second argument : True if overwrite possible

### Header generation

In [5]:
# Header block addition

headerbloc_content = ['UNB+UNOA:3+::ABC10779001+::HEXAGON ABC+' + datatransfertdict['today'][2:8] + ':1245+' + datatransfertdict['payref'] + '\'',
'UNH+1+PAYMUL:D:96A:UN:FUN01G\'',
'BGM+452+' + datatransfertdict['payref']+'+9\'',
'DTM+137:'+datatransfertdict['paydate']+':102\'',
'LIN+1\'',
'DTM+203:'+datatransfertdict['paydate']+':102\'',
'RFF+AEK:'+ datatransfertdict['paynotes']+'\'',
'MOA+9:'+transfertsum+':GBP\'',
'FII+OR+'+datatransfertdict['payacc']+':154:133+GB\'',
'NAD+OY++THE STATE51 CONSPIRACY:17 HEREFORD STREET :E2 6EX LONDON:UNITED KINGDOM\'']
                      
for i in range(len(headerbloc_content)):
    paymuldata.write(i,0,headerbloc_content[i])

### Content blocks generation

In [6]:
# Blocks addition methods

def transfert_line_treatment(line_num): #int, Excel row 
    returnlist =[]
    datalist = sh.row_values(line_num)
    
    payeename = str(datalist[0])[:17]
    amount = str(datalist[1])
    bankdetail = str(datalist[2])
    sortcode = bankdetail[:6]
    accountnumber = bankdetail[-8:]
    
    returnlist.append('SEQ++'+str(line_num)+'\'')
    returnlist.append('MOA+9:'+amount+':GBP\'')
    returnlist.append('RFF+CR: '+datatransfertdict['uniqref']+'\'')
    returnlist.append('RFF+PQ: '+datatransfertdict['uniqref']+'\'')
    returnlist.append('PAI+::2\'')
    returnlist.append('FCA+13\'')
    returnlist.append('FII+BF+'+accountnumber+':'+payeename+'+:::'+sortcode+':154:133+GB\'')
    returnlist.append('NAD+BE++'+payeename+': : :UNITED KINGDOM\'')

    return returnlist

def add_lines(first, content): #int, int, list
    for i in range(len(content)):
        paymuldata.write(first+i, 0, content[i])

In [7]:
# Blocks addition main

for i in range(sh.nrows-1):
    add_lines(10+8*i,transfert_line_treatment(i+1)) # add blocks after header

### Footer block generation

In [8]:
nbrows = sh.nrows - 1
footerstartline = 10 + 8* nbrows

paymuldata.write(footerstartline, 0, 'CNT+39:'+str(nbrows)+'\'')
paymuldata.write(footerstartline+1, 0, 'UNT+475+1\'')
paymuldata.write(footerstartline+2, 0, 'UNZ+1+'+datatransfertdict['payref']+'\'')

### Output file .xls format generation

In [9]:
# Save object in a native format Excel file
outputbook.save('Paymul_output.xls')

## Output file controls